# Multisignature and P2SH (pay to script hash) scripts in Bitcoin

## 1. Multisignature script

In [1]:
import bitcoin  #pybitcointools
from itertools import product

In [2]:
#Initializing

M = 2 #Minimum number of signatures to approve the transaction (quorum)
N = 3 #Number of all the signatures that can be provided
priv_keys = [bitcoin.random_key() for _ in range(N)]
pub_keys = [bitcoin.privkey_to_pubkey(priv_key) for priv_key in priv_keys]
tx = 'Transaction example'
tx_hash = bitcoin.txhash(tx)

In [3]:
#Locking script building

locking_script = '{M} {pub_keys[0]} {pub_keys[1]} {pub_keys[2]} {N} CHECKMULTISIG'.format(M=M,
                                                                                          pub_keys=pub_keys,
                                                                                          N=N)
print('Multisig locking script:', locking_script, sep='\n\n')

Multisig locking script:

2 04dd088448817e42324919162d769175ce7b5757845df8b1ea10c6709fdadb624c8fa1c8297e240a0957930facbd60cdb0248f40b03d098d76217e7293d4cc7147 04f37f1ff34a693eae99839379fb6ae8196b43d4c0dbc7e3eaa34d279fcba445558b97359fb12563323b67c3664f312c5d48bd77fdd2904e977567c68194173857 047a56b52936a7e750fffd04735b7ffcf529e084c086838ca764caacf0124e49b60f1a4eb4a97ea414f6b86b1742c8e5567087bef4a2878fa6ff587a119f3022a2 3 CHECKMULTISIG


In [4]:
#Making signatures (i.e. unlocking scripts)

sigs = [bitcoin.ecdsa_raw_sign(tx_hash, priv_key) for priv_key in priv_keys]
sigs
der_encoded_sigs = [bitcoin.der_encode_sig(*sig) for sig in sigs]
for i in range(N):
    print('Signature {i}: {sig}'.format(i=i, sig=der_encoded_sigs[i]), end='\n\n')

Signature 0: 30440220497545ea68e7bf0b1e56ad6f26cc523f473409e282580876d33c75161078daf00220481abf8248a5ef77416a649e2e28a23d07e551c0d491b37b72aa4151685532b6

Signature 1: 304402206ac21ee51ef38c373b7820e352bba444cba7b328b5af44366708f393756e852602204d5142596354f0826703a9e9c84ce3fa9d2c83c90ff0058f96238e675ea34c4a

Signature 2: 304402207c21a86ed5838295c3d68056ea07f4d110803ef59155c0018ff3a6439eb346c002207736ae358132ad92965ac7bdda0378fcbdd45c483864a384f6816d12925f7c1c



In [5]:
#Unlocking script building
#Starting with 0 by convention

unlocking_script = '0 {sigs[1]} {sigs[2]}'.format(sigs=der_encoded_sigs)
print('Multisig unlocking script:', unlocking_script, sep='\n')

Multisig unlocking script:
0 304402206ac21ee51ef38c373b7820e352bba444cba7b328b5af44366708f393756e852602204d5142596354f0826703a9e9c84ce3fa9d2c83c90ff0058f96238e675ea34c4a 304402207c21a86ed5838295c3d68056ea07f4d110803ef59155c0018ff3a6439eb346c002207736ae358132ad92965ac7bdda0378fcbdd45c483864a384f6816d12925f7c1c


In [6]:
#Pushing both scripts to a stack
#Comparing each signature with each public key until reaching M successful verifications

def multisig_verify(lock, unlock, tx_hash):
    stack = unlock.split() + lock.split()
    if stack.pop() == 'CHECKMULTISIG':
        N = int(stack.pop())
        pub_keys = []
        sigs = []
        for _ in range(N):
            pub_keys.append(stack.pop())
        M = int(stack.pop())
        for _ in range(M+1):
            sigs.append(stack.pop())
        sigs = [bitcoin.der_decode_sig(sig) for sig in sigs]
        n_verified = 0
        multi_verified = False
        for sig, pub_key in product(sigs, pub_keys):
            if bitcoin.ecdsa_raw_verify(tx_hash, sig, pub_key):
                n_verified += 1
            if n_verified == M:
                multi_verified = True
                break
        return multi_verified
    
if multisig_verify(locking_script, unlocking_script, tx_hash):
    print('Successfully verified!')

Successfully verified!


## 2. P2SH script

In [7]:
#Constructing P2SH script

#Arranging data types
splitted = locking_script.split()
splitted[0] = int(splitted[0]) 
splitted[4] = int(splitted[4])
splitted[5] = 174  #CHECKMULTSIG opcode

#Serializing and hashing
serialized = bitcoin.serialize_script(splitted)
redeem_hash = bitcoin.ripemd160(bitcoin.sha256(serialized))
p2sh_locking = 'HASH160 {} EQUAl'.format(redeem_hash)
print('P2SH script:', p2sh_locking, sep='\n\n')

P2SH script:

HASH160 f1cddc958ce454f0ebf205fd3fed85e5b2c3dc61 EQUAl


In [8]:
#P2SH address (version prefix 5)
p2sh_address = bitcoin.hex_to_b58check(redeem_hash, magicbyte=5)
print('P2SH address:', p2sh_address, sep='\n\n')

P2SH address:

3PjZPHypGh4bybzbBgDqT2yyUaXBghzyWd
